ChEn-3170: Computational Methods in Chemical Engineering Fall 2018 UMass Lowell; Prof. V. F. de Almeida **13Nov2018**

# 12. Linear Least-Squares Fitting w/ Fourier Basis Functions
$  
  \newcommand{\Amtrx}{\boldsymbol{\mathsf{A}}}
  \newcommand{\Bmtrx}{\boldsymbol{\mathsf{B}}}
  \newcommand{\Mmtrx}{\boldsymbol{\mathsf{M}}}
  \newcommand{\Imtrx}{\boldsymbol{\mathsf{I}}}
  \newcommand{\Pmtrx}{\boldsymbol{\mathsf{P}}}
  \newcommand{\Lmtrx}{\boldsymbol{\mathsf{L}}}
  \newcommand{\Umtrx}{\boldsymbol{\mathsf{U}}}
  \newcommand{\Smtrx}{\boldsymbol{\mathsf{S}}}
  \newcommand{\xvec}{\boldsymbol{\mathsf{x}}}
  \newcommand{\avec}{\boldsymbol{\mathsf{a}}}
  \newcommand{\bvec}{\boldsymbol{\mathsf{b}}}
  \newcommand{\cvec}{\boldsymbol{\mathsf{c}}}
  \newcommand{\rvec}{\boldsymbol{\mathsf{r}}}
  \newcommand{\mvec}{\boldsymbol{\mathsf{m}}}
  \newcommand{\gvec}{\boldsymbol{\mathsf{g}}}
  \newcommand{\zerovec}{\boldsymbol{\mathsf{0}}}
  \newcommand{\norm}[1]{\bigl\lVert{#1}\bigr\rVert}
  \newcommand{\transpose}[1]{{#1}^\top}
  \DeclareMathOperator{\rank}{rank}
$

---
## Table of Contents
* [Introduction](#intro)
* [Fourier Expansion](#fexp)
* [Experimental Data](#data)
* [Linear System](#ls)
* [LS Data Fitting](#lsdf)
* [Modal Analysis](#modal)
* [Power Spectrum](#spectrum)
---

## Introduction<a id="intro"></a>
The least-squares method with Fourier basis functions is a powerful computational tool for data fitting and data analysis, course notes are stored [here](https://studentuml-my.sharepoint.com/:o:/g/personal/valmor_dealmeida_uml_edu/EjTsbl-lFaxIq02j738gSUIBbReQkfYbTfqmLh1GiXD_cA?e=0oPQ8h). 

## Fourier Expansion<a id="fexp"></a>

The Fourier expansion for approximating a function is

\begin{equation*}
b(t) = \sum\limits_{k=0}^N \alpha_k\,\cos(k\,\mu\,t) + \beta_k\,\sin(k\,\mu\,t)
\end{equation*}

it is a particular form of the generic linear combination of functions ${f_j}(t)$

\begin{equation*}
b(t) = \sum\limits_{j=1}^{2N+1} x_j\,f_j(t) .
\end{equation*}

If we have a set of values of the independent variable $t_i, i=1,\ldots,m$, the above Fourier expression when applied to every $t_i$ gives

where $\Amtrx =  \begin{pmatrix}
1 & \cos(\mu\,t_1) & \sin(\mu\,t_1) & \cos(2\mu\,t_1) & \sin(2\mu\,t_1) & \ldots & \cos(N\mu\,t_1) & \sin(N\mu\,t_1)  \\
1 & \cos(\mu\,t_2) & \sin(\mu\,t_2) & \cos(2\mu\,t_2) & \sin(2\mu\,t_2) & \ldots & \cos(N\mu\,t_2) & \sin(N\mu\,t_2)  \\
\vdots  & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
1 & \cos(\mu\,t_m) & \sin(\mu\,t_m) & \cos(2\mu\,t_m) & \sin(2\mu\,t_m) & \ldots & \cos(N\mu\,t_m) & \sin(N\mu\,t_m)  \\
 \end{pmatrix}$, 
 $\xvec =  \begin{pmatrix}
  \alpha_0 \\ 
  \alpha_1 \\
  \beta_1 \\
  \vdots \\
  \alpha_N \\
  \beta_N \\ 
 \end{pmatrix}$, 
and 
$\bvec = \begin{pmatrix}
 b_1 \\ 
 b_2 \\ 
 \vdots  \\ 
 b_m \\ 
\end{pmatrix} $.

## Experimental Data<a id="data"></a>
Data for this lecture is found in the `data/` directory of the course [repository](https://github.com/dpploy/chen-3170/tree/master/notebooks/data). The data is organized in two columns of NO$_2$ mass concentration ($\mu$g/m$^3$) versus time (h); see `data/urban-no2.dat`.

In [ ]:
'''Function: read experimental data'''

def read_experimental_data( filename ):
    import io                     # import io module
    finput = open(filename, 'rt') # create file object

    import numpy as np

    for line in finput:
    
        line = line.strip()
    
        if line[0] == '#': # skip comments in the file
            continue
        
        var_line = line.split(' = ')
    
        if var_line[0] == 'n_pts':
            n_pts = int(var_line[1])
            time_expt = np.zeros(n_pts)
            no2_mass_cc_expt = np.zeros(n_pts)
            idx = 0 # counter
        else:
            data = line.split(' ')
            time_expt[idx]       = float(data[0])
            no2_mass_cc_expt[idx] = float(data[1])
            idx += 1
            
    return (n_pts, time_expt, no2_mass_cc_expt)

In [ ]:
'''Read experimental data'''

import numpy as np
(n_pts, time_expt, no2_mass_cc_expt) = read_experimental_data('data/urban-no2.dat')
        
print('m = ',n_pts)
np.set_printoptions(precision=2)
print('time  [h]      =',time_expt)
print('NO2   [ug/m^3] =', no2_mass_cc_expt)

In [ ]:
'''Function: plot experimental data'''

def plot_experimental_data( time_expt, no_mass_cc_vec ):
    
    import matplotlib.pyplot as plt

    plt.figure(1, figsize=(7, 7))

    plt.plot(time_expt, no_mass_cc_vec,'r*',label='experimental')
    
    plt.xlabel(r'Time [h]',fontsize=18)
    plt.ylabel(r'NO$_2$ [$\mu$g/m$^3$]',fontsize=18)
    plt.title('Release of NO$_2$ in Urban Area',fontsize=20)
    plt.legend(loc='best',fontsize=12)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.grid(True)
    plt.show()
    print('')
    
    return

In [ ]:
'''Plot experimental data'''
plot_experimental_data(time_expt, no2_mass_cc_expt)

### Linear System<a id="ls"></a>
Let's build the matrix system $\Amtrx\,\xvec = \bvec$

where $\Amtrx =  \begin{pmatrix}
1 & \cos(\mu\,t_1) & \sin(\mu\,t_1) & \cos(2\mu\,t_1) & \sin(2\mu\,t_1) & \ldots & \cos(N\mu\,t_1) & \sin(N\mu\,t_1)  \\
1 & \cos(\mu\,t_2) & \sin(\mu\,t_2) & \cos(2\mu\,t_2) & \sin(2\mu\,t_2) & \ldots & \cos(N\mu\,t_2) & \sin(N\mu\,t_2)  \\
\vdots  & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
1 & \cos(\mu\,t_m) & \sin(\mu\,t_m) & \cos(2\mu\,t_m) & \sin(2\mu\,t_m) & \ldots & \cos(N\mu\,t_m) & \sin(N\mu\,t_m)  \\
 \end{pmatrix}$, 
 $\xvec =  \begin{pmatrix}
  \alpha_0 \\ 
  \alpha_1 \\
  \beta_1 \\
  \vdots \\
  \alpha_N \\
  \beta_N \\ 
 \end{pmatrix}$, 
and 
$\bvec = \begin{pmatrix}
 b_1 \\ 
 b_2 \\ 
 \vdots  \\ 
 b_m \\ 
\end{pmatrix} $.

In [ ]:
'''Build the matrix system'''

import math

period = np.max(time_expt)  # [h]
omega  = 1/period          # cycle frequency [1/h]
mu     = 2*math.pi * omega # radian frequency [rad/h]

n_pairs = 3

try:
    from chen_3170.toolkit import build_fourier_matrix
except ModuleNotFoundError:
    assert False, 'You need to provide your own function to build the linear system. Bailing out.'

a_mtrx = build_fourier_matrix( mu, n_pairs, time_expt )

b_vec = no2_mass_cc_expt

#from chen_3170 import toolkit
#rank = toolkit.matrix_rank(a_mtrx)
rank = np.linalg.matrix_rank(a_mtrx)
print('rank(A) =',rank)
assert rank == 2*n_pairs+1

### LS Data Fitting w/ Fourier Basis Functions<a id="lsdf"></a>

Let's build the LS normal equations and solve

\begin{equation*}
\Amtrx^\top\Amtrx\,\xvec_\text{LS}  = \Amtrx^\top\,\bvec .
\end{equation*}

The least squares solution $\xvec_\text{LS}$ gives the optimal values of the Fourier modes, $\alpha_k\,\cos(k\,\mu\,t)$ and $\beta_k\,\sin(k\,\mu\,t)$.

In [ ]:
'''Solve the normal equations for the Fourier amplitudes'''

#x_vec = toolkit.solve( a_mtrx.transpose()@a_mtrx, a_mtrx.transpose()@b_vec )
x_vec = np.linalg.solve( a_mtrx.transpose()@a_mtrx, a_mtrx.transpose()@b_vec )

In [ ]:
'''Function: plot the LS Fourier fit and all modes'''

def plot_fourier_fit(time_expt, no2_mass_cc_expt, mu, x_vec):
    
    import matplotlib.pyplot as plt
    
    plt.figure(2, figsize=(7,7))

    # plot experimental data
    plt.plot(time_expt, no2_mass_cc_expt,'r*',label='experimental')

    # plot LS Fourier fitting
    n_plot_pts = 100
    time_plot = np.linspace(time_expt[0],time_expt[-1],n_plot_pts)
    dummy = 0
    a_mtrx = build_fourier_matrix (mu, n_pairs, time_plot )
    no2_mass_cc_plot = a_mtrx @ x_vec
    
    plt.plot( time_plot, no2_mass_cc_plot,'b-',label='LS Fourier fitting' )

    color_ids = np.linspace(0,1,a_mtrx.shape[1])
    for (j,color_id) in zip(range(a_mtrx.shape[1]),color_ids):

        color=plt.cm.nipy_spectral(color_id)
        
        if j == 0:
            color='black'
            k = 0
            label=r'$\alpha_{%i}$=%4.2e'%(k,x_vec[j])
        elif j%2 == 0:
            k = j/2
            label=r'$\beta_{%i}$=%4.2e sin($\mu t$)'%(k,x_vec[j])
        else:
            k = (j+1)/2
            label=r'$\alpha_{%i}$=%4.2e cos($\mu t$)'%(k,x_vec[j])
        
        vertical_offset = 50 # to improve visibility of modes
        if j == 0:
            vertical_offset = 0
            
        plt.plot(time_plot, x_vec[j]*a_mtrx[:,j]-vertical_offset,label=label,color=color )
        
    plt.xlabel(r'Time [h]',fontsize=18)
    plt.ylabel(r'NO$_2$ [$\mu$g/m$^3$]',fontsize=18)
    plt.title('Release of NO$_2$ in Urban Area',fontsize=20)

    (x_min,x_max) = plt.xlim()
    dx = abs(x_max-x_min)
    x_text = x_min + dx*0.25
    
    (y_min,y_max) = plt.ylim()
    dy = abs(y_max-y_min)
    y_text = y_min + dy*0.02
    
    plt.text(x_text, y_text, 
             r'$\mu=$%8.2e [rad/h],   $N$=%i'%
             (mu,n_pairs),fontsize=16)

    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(loc='upper right',bbox_to_anchor=(1.5, 1),fontsize=12)
    plt.grid(True)
    plt.show()
    print('')
    return

In [ ]:
'''Plot the LS Fourier fit'''

plot_fourier_fit(time_expt, no2_mass_cc_expt, mu, x_vec)

### Modal Analysis<a id="modal"></a>

Vary the number of modes to observe the quality of fitting. There is much to investigate here but in this course we will be limited to an *eyeball* analysis and investigation of the rank of $\Amtrx$ as the Fourier basis increases in size.

### Power Spectrum<a id="spectrum"></a>

Power spectrum is the average amplitude of the modes as a function of frequency or the Fourier transform of the data. Compute the power as follows

\begin{equation*}
P(\omega_k) = \sqrt{\frac{\alpha_k^2 + \beta_k^2}{4}}
\end{equation*}

where the cycle frequency for each mode is 

\begin{equation*}
\omega_k = \frac{k\,\mu}{2\pi}
\end{equation*}


In [ ]:
'''Function: plot the power spectrum (Fourier transform)'''

def plot_power_spectrum(period,mu, x_vec):
    
    import matplotlib.pyplot as plt
    
    fig, ax = plt.subplots(figsize=(20,6))
    
    import numpy as np
    power = np.zeros( int((x_vec.size-1)/2) + 1 )
    power[0] = x_vec[0]
    
    import math
    
    for k in range(power.size):
        if k == 0:
            continue
        beta_id  = 2*k
        alpha_id = beta_id - 1
        
        alpha = x_vec[alpha_id]
        beta  = x_vec[beta_id]
        
        power[k] = math.sqrt( (alpha**2 + beta**2)/ 4 )  
    
    if power.size == 2:
        plt.bar( range(power.size), power, color='orange', width=0.25 )
    else:
        plt.bar( list(range(power.size))[1:], power[1:], 
                 color='orange', width=0.5 )
    
    ax.set_ylabel('Power [$\mu$g/m$^3$]',fontsize=18)
    ax.set_xlabel('Cycle Frequency ($k=T\omega_k=k\mu/(2\pi)$) [cycles/day]',fontsize=18)
    ax.xaxis.grid( True, linestyle='-', which='major', color='lightgrey',alpha=0.9)
    fig.suptitle('Power Spectrum $N=$%i'%k, fontsize=20)
    plt.show()

    print('')
    return

In [ ]:
for n_pairs in range(1,13,2):

    a_mtrx = build_fourier_matrix( mu, n_pairs, time_expt )
    
    #from chen_3170 import toolkit
    #rank = toolkit.matrix_rank(a_mtrx)
    rank = np.linalg.matrix_rank(a_mtrx)
    print('rank(A) =',rank)
    assert rank == 2*n_pairs+1,'The LS normal matrix is rank deficient; matrix shape: (%r,%r)'%(a_mtrx.shape[0],a_mtrx.shape[1])
    
    #x_vec = toolkit.solve( a_mtrx.transpose()@a_mtrx, a_mtrx.transpose()@b_vec )
    x_vec = np.linalg.solve( a_mtrx.transpose()@a_mtrx, a_mtrx.transpose()@b_vec )
    plot_fourier_fit(time_expt, no2_mass_cc_expt, mu, x_vec)
    plot_power_spectrum(period,mu, x_vec)

## Interactive Results <a id="ires"></a>

In [ ]:
'''Create interactive plot'''

import plotly.offline as py
py.init_notebook_mode()
#plotly.offline.plot(include_mathjax='cdn')
import plotly.graph_objs as go
from ipywidgets import interact

fig = go.FigureWidget()

fig.layout.titlefont.size = 22
fig.layout.titlefont.family = 'Rockwell'

In [ ]:
from plotly import __version__
print( __version__) # requires version >= 1.9.0

In [ ]:
'''Vary number of modes'''

@interact( n_modes=(1, 10, 1), select=['LS Fourier Fit','w/ Modes'] )
def update(n_modes=1, select='Fit'):
    with fig.batch_update():
        
        layout = go.Layout(title='Release of NO$_2$ in Urban Area', 
                 xaxis={'title':'Time [h]'}, 
                 yaxis=dict(side='left',title='$NO2$ [ug/m^3]')
                          )
        fig.layout = layout
        
        a_mtrx = build_fourier_matrix mu, n_modes, time_expt )
        x_vec  = np.linalg.solve( a_mtrx.transpose()@a_mtrx, a_mtrx.transpose()@b_vec )
        
        def plotly_experimental_data(fig):
            # plot experimental data
            scatt = fig.add_scatter(name="Exp't",mode='markers')
            scatt.x = time_expt
            scatt.y = no2_mass_cc_expt
            scatt.marker.color='red'
            scatt.marker.symbol='star'
            scatt.marker.size=10
            return
        
        def plotly_fourier_fit(fig,modes=False):
            # plot LS Fourier fitting 
            n_plot_pts = 180
            time_plot = np.linspace(time_expt[0],time_expt[-1],n_plot_pts)
            dummy = 0
            a_mtrx = build_fourier_matrix( mu, n_modes, time_plot )
            no2_mass_cc_plot = a_mtrx @ x_vec
            
            scatt = fig.add_scatter(name='LSF Fit',mode='lines')
            scatt.x = time_plot
            scatt.y = no2_mass_cc_plot
            scatt.line.color = 'blue'
            
            if modes == False:
                return
            
            import matplotlib.pyplot as plt
            color_ids = np.linspace(0,1,a_mtrx.shape[1])
            for (j,color_id) in zip(range(a_mtrx.shape[1]),color_ids):
                               
                color=plt.cm.nipy_spectral(color_id)
                red_channel = str(int(color[0]*255))
                green_channel = str(int(color[1]*255))
                blue_channel = str(int(color[2]*255))
                color='rgb('+red_channel+','+green_channel+','+blue_channel+')'
        
                if j == 0:
                    color='black'
                    k = 0
                    label='$\alpha_{%i}$=%4.2e'%(k,x_vec[j])
                elif j%2 == 0:
                    k = j/2
                    label='$\beta_{%i}$=%4.2e sin($\mu t$)'%(k,x_vec[j])
                else:
                    k = (j+1)/2
                    label='$\alpha_{%i}$=%4.2e cos($\mu t$)'%(k,x_vec[j])
        
                scatt      = fig.add_scatter(name=label)
                
                vertical_offset = 50 # to improve visibility of modes
                if j == 0:
                    vertical_offset = 0
                    
                scatt.x = time_plot
                scatt.y = x_vec[j]*a_mtrx[:,j]-vertical_offset
                scatt.line.color = color
                
            return           
        
        if select == 'LS Fourier Fit':
            
            fig.data = [] # clear the data stack
                
            # plot experimental data
            plotly_experimental_data(fig)
           
            # plot LS Fourier fitting 
            plotly_fourier_fit(fig)
      
        elif select == 'w/ Modes':
            
            fig.data = [] # clear the data stack
                        
            # plot experimental data
            plotly_experimental_data(fig)
           
            # plot LS Fourier fitting 
            plotly_fourier_fit(fig,modes=True)
        
        else:
            assert False, 'no other option; bail out'
fig